In [1]:
import torch
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import string
import time
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

%load_ext autoreload
%autoreload 2

from rnn import RNN, PrepareData

In [2]:
#input_dim = 7
hidden_dim = 30
num_layers = 2
seq_size = 25
all_letters = string.printable
n_letters = len(all_letters)

In [3]:
#dataset = PrepareData('input.txt', seq_size, val_ratio=0.2)
dataset = PrepareData('sample-music2.txt', seq_size)
train_data, val_data = dataset.dataset()

In [51]:
def inputTensor(line):
    if len(line) == 1:
        tensor = torch.zeros(1, n_letters)
        tensor[0, all_letters.find(line[0])] = 1
    else:
        tensor = torch.zeros(len(line)-1, n_letters)
        for idx, letter in enumerate(line[:-1]):
            tensor[idx, all_letters.find(letter)] = 1
    return Variable(tensor.cuda())
        
def targetTensor(line):
    letter_indexes = [all_letters.find(letter) for letter in line[1:]]
    return Variable(torch.LongTensor(letter_indexes).cuda())

def save_model(model, path):
    torch.save(model.state_dict(), path)
    
def load_model(model, path):
    model.load_state_dict(torch.load(path))

In [5]:
model = RNN(n_letters, hidden_dim, num_layers=num_layers, output_size=n_letters)
model = model.cuda()
criterion = nn.CrossEntropyLoss()

In [59]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [60]:
n_epochs = 20
all_losses = []

print_every = 500
init_hidden_every = 20

start_time = time.time()

for epoch in range(n_epochs):
    train_cnt = 0
    total_loss = 0.
    
    model.hidden = model.init_hidden()
    
    #dataset.shuffle_data()
    #train_data, val_data = dataset.dataset()
    
    for sentence in train_data:
        model.zero_grad()
        
        loss = 0.
        inputs = inputTensor(sentence)
        targets = targetTensor(sentence)
        
        preds = model(inputs)
        loss = criterion(preds, targets)
            
        loss.backward(retain_graph=True)
        optimizer.step()
        total_loss += loss
        train_cnt += 1
        
        if train_cnt % init_hidden_every == 0:
            model.hidden = model.init_hidden()
        
        if train_cnt % print_every == 0:
            total_loss /= print_every
            all_losses.append(total_loss)
            uptime = time.time() - start_time
            print('%dm:%s -- [%d, %d], loss: %f' % (uptime/60, uptime%60, epoch, train_cnt, total_loss))
            total_loss = 0
#scheduler.step()

0m:11.707715272903442 -- [0, 500], loss: 1.967980
0m:22.397526264190674 -- [0, 1000], loss: 1.957015
0m:40.806588888168335 -- [1, 500], loss: 1.957506
0m:51.580753803253174 -- [1, 1000], loss: 1.913785
1m:9.764806747436523 -- [2, 500], loss: 1.908384
1m:20.579535484313965 -- [2, 1000], loss: 1.846005
1m:39.05263662338257 -- [3, 500], loss: 1.922634
1m:49.94528126716614 -- [3, 1000], loss: 1.826035
2m:8.250165700912476 -- [4, 500], loss: 1.854512
2m:19.01862144470215 -- [4, 1000], loss: 1.773407
2m:37.65753698348999 -- [5, 500], loss: 1.817026
2m:48.24101638793945 -- [5, 1000], loss: 1.749392
3m:6.459793567657471 -- [6, 500], loss: 1.795434
3m:17.06702995300293 -- [6, 1000], loss: 1.719495
3m:35.34467554092407 -- [7, 500], loss: 1.779078
3m:49.22354006767273 -- [7, 1000], loss: 1.691813
4m:12.409470319747925 -- [8, 500], loss: 1.746116
4m:24.02322292327881 -- [8, 1000], loss: 1.668914
4m:41.13911437988281 -- [9, 500], loss: 1.731472
4m:50.33426809310913 -- [9, 1000], loss: 1.649536
5m:6

In [43]:
def test(start_with, max_length):
    model.hidden = model.init_hidden()
    seq = str(start_with)
    idx = []
    inputs = inputTensor(seq[-1])
    
    for i in range(max_length):
        output = model(inputs)
        _, pred = torch.max(output.data, 1)
        
        seq += all_letters[pred[0]]
        idx.append(pred[0])
        inputs = inputTensor(letter)
    return seq, idx

In [61]:
test_seq = train_data[5]

seq, idx = test(test_seq[7],25)
print('Generated sequence: %s' % (seq))
print('Sequence index: ', idx)
print('Test sequence: %s' % test_seq)

Generated sequence: M GABAAAAFFAAFAAFFAAFFAFFA
Sequence index:  [94, 42, 36, 37, 36, 36, 36, 36, 41, 41, 36, 36, 41, 36, 36, 41, 41, 36, 36, 41, 41, 36, 41, 41, 36]
Test sequence: ree.fr
M: 4/4
L: 1/8
Q:1/


In [62]:
save_model(model, 'sample-model.pth')

In [ ]:
plt.figure()
plt.plot(all_losses)